In [ ]:
import numpy as np
import caffe

caffe_root = './'  # this file is expected to be in {caffe_root}

In [ ]:
caffe.set_mode_cpu()
net = caffe.Net(caffe_root + 'examples/mnist/lenet_train_test.prototxt',
                caffe_root + 'examples/mnist/lenet_iter_10000.caffemodel',
                caffe.TEST)

In [ ]:
# import scipy.io

# mdict = {}
# for lyr in net.params:
#     mdict['Wval' + lyr] = net.params[lyr][0].data

# scipy.io.savemat('ToMatlab.mat', mdict)

In [ ]:
import copy

W = {}
b = {}
for lyr in net.params:
    W[lyr] = copy.deepcopy(net.params[lyr][0].data)
    b[lyr] = copy.deepcopy(net.params[lyr][1].data)

In [ ]:
from nnS_lib import *
step = 0.1
P = np.arange(0, 1 + step/2, step)

Acc = []
for p in P:
    print p, 
    for lyr in net.params:
        net.params[lyr][0].data[...] = dropOutputwise(W[lyr], p)
        #    net.params[lyr][1].data[...] = dropLow(b[lyr], p)
    acc = []
    for i in range(100):
        temp = net.forward()['accuracy']
        acc.append(float(temp))
    Acc_p = np.mean(np.asarray(acc))
    Acc.append(Acc_p)
    print Acc_p

In [ ]:
import matplotlib.pyplot as plt, mpld3

%matplotlib inline
# mpld3.enable_notebook()
# mpld3.disable_notebook()
fig = plt.figure()
fig = plt.plot(P, Acc)